<a class="anchor" id="0.1"></a>
# **Table of Contents**
1. [Introduction](#1)
2. [Changelog](#2)
3. [Business Understanding](#3)
4. [Data Understanding](#4)
5. [Data Preperation](#5)
6. [Modeling](#6)
7. [Evaluation](#7)
8. [Improvements](#8)
9. [Conclusion](#9)

# **1. Introduction** <a class="anchor" id="1"></a>
[Table of Contents](#0.1)

This notebook will explore k Nearest Neighbour by going through the CRISP-DM process. The goal is to gain insight of how to use k-NN in Python and its usefulness.

# **2. Changelog** <a class="anchor" id="2"></a>
[Table of Contents](#0.1)

# **3. Business Understanding** <a class="anchor" id="3"></a>
[Table of Contents](#0.1)

# **4. Data Understanding** <a class="anchor" id="4"></a>
[Table of Contents](#0.1)

# **5. Data Preperation** <a class="anchor" id="5"></a>
[Table of Contents](#0.1)

# **6. Modeling** <a class="anchor" id="6"></a>
[Table of Contents](#0.1)

# **7. Evaluation** <a class="anchor" id="7"></a>
[Table of Contents](#0.1)

# **8. Improvements** <a class="anchor" id="8"></a>
[Table of Contents](#0.1)

# **9. Conclusion** <a class="anchor" id="9"></a>
[Table of Contents](#0.1)